# SQLTemple-1.1B-Alpha Training Pipeline

This notebook implements the complete end-to-end training pipeline for SQLTemple-1.1B-Alpha following a simplified approach for an alpha version.

- **Model**: TinyLlama-1.1B-Chat-v1.0 (base)
- **Dataset**: Spider
- **Method**: LoRA fine-tuning
- **Output**: GGUF-ready model for C++ runtime

---

## Setup and Imports

In [1]:
%pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 MB 23.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.4 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.16-cp312-cp312-linux_x86_64.whl size=4503249 sha256=bd7e64c1ba8c829a158f4f0fdaa57e810a8cb8d2275135e1990f1b616504e930
  Stored in directory: /root/.cache/pip/wheels/90/82/ab/8784ee3fb99ddb07fd36a679ddbe63122cc07718f6c1eb3be8
Successfully built llama-cpp-python


In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
import torch
import json
import os
from datetime import datetime

os.environ["WANDB_DISABLED"] = "true"

print("Starting SQLTemple-1.1B Training Pipeline")
print(f"Started at: {datetime.now()}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

Starting SQLTemple-1.1B Training Pipeline
Started at: 2025-08-20 20:45:44.953238
PyTorch version: 2.8.0+cu126
CUDA available: True
GPU: Tesla T4
GPU Memory: 15.8 GB


## Dataset Loading

Loading Spider dataset for SQL training.

In [3]:
print("Loading Spider dataset...")
spider = load_dataset("xlangai/spider", split="train")

print(f"Spider: {len(spider):,} examples")
print("Sample Spider example:")
print(json.dumps(spider[0], indent=2)[:500] + "...")

Loading Spider dataset...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

spider/train-00000-of-00001.parquet:   0%|          | 0.00/831k [00:00<?, ?B/s]

spider/validation-00000-of-00001.parquet:   0%|          | 0.00/126k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1034 [00:00<?, ? examples/s]

Spider: 7,000 examples
Sample Spider example:
{
  "db_id": "department_management",
  "query": "SELECT count(*) FROM head WHERE age  >  56",
  "question": "How many heads of the departments are older than 56 ?",
  "query_toks": [
    "SELECT",
    "count",
    "(",
    "*",
    ")",
    "FROM",
    "head",
    "WHERE",
    "age",
    ">",
    "56"
  ],
  "query_toks_no_value": [
    "select",
    "count",
    "(",
    "*",
    ")",
    "from",
    "head",
    "where",
    "age",
    ">",
    "value"
  ],
  "question_toks": [
    "How",
    ...


## Tokenizer Setup

Loading TinyLlama tokenizer and configuring for chat format.

In [4]:
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0", use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

print("Tokenizer loaded")
print(f"Vocab size: {tokenizer.vocab_size:,}")
print(f"EOS token: '{tokenizer.eos_token}' (ID: {tokenizer.eos_token_id})")
print(f"PAD token: '{tokenizer.pad_token}' (ID: {tokenizer.pad_token_id})")

Loading tokenizer...


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Tokenizer loaded
Vocab size: 32,000
EOS token: '</s>' (ID: 2)
PAD token: '</s>' (ID: 2)


##Data Preprocessing

Converting Spider dataset into instruction format for chat-style training.

In [5]:
def preprocess_spider(example):
    """Convert Spider example to instruction format"""
    question = example["question"]
    sql = example["query"]
    db_id = example.get("db_id", "")

    # For now, we'll work without explicit schema since it's not in the training split
    # We can include the database ID as context
    if db_id:
        user_prompt = f"Database: {db_id}\nQuestion: {question}"
    else:
        user_prompt = f"Question: {question}"

    prompt = f"<|system|>You are an SQL assistant. Answer in valid SQL.\n<|user|>{user_prompt}\n<|assistant|>"
    return {"question": question, "sql": sql, "prompt": prompt, "db_id": db_id}

print("Testing preprocessing function...")
sample_spider = preprocess_spider(spider[0])

print("Spider formatted example:")
print(f"Prompt: {sample_spider['prompt'][:200]}...")
print(f"SQL: {sample_spider['sql']}")
print(f"DB ID: {sample_spider['db_id']}")

Testing preprocessing function...
Spider formatted example:
Prompt: <|system|>You are an SQL assistant. Answer in valid SQL.
<|user|>Database: department_management
Question: How many heads of the departments are older than 56 ?
<|assistant|>...
SQL: SELECT count(*) FROM head WHERE age  >  56
DB ID: department_management


In [6]:
def preprocess_example(example):
    """Preprocess a single Spider example for causal language modeling"""
    processed = preprocess_spider(example)

    prompt = processed["prompt"]
    sql = processed["sql"]

    full_text = prompt + sql + tokenizer.eos_token

    tokenized = tokenizer(
        full_text,
        truncation=True,
        max_length=512,
        padding="max_length",
        return_tensors=None
    )

    input_ids = tokenized["input_ids"]
    attention_mask = tokenized["attention_mask"]

    labels = input_ids.copy()

    prompt_tokenized = tokenizer(
        prompt,
        truncation=True,
        max_length=512,
        padding="max_length",
        return_tensors=None
    )

    prompt_length = len([token for token in prompt_tokenized["input_ids"] if token != tokenizer.pad_token_id])

    for i in range(min(prompt_length, len(labels))):
        labels[i] = -100

    for i in range(len(labels)):
        if attention_mask[i] == 0:
            labels[i] = -100

    return {
        "input_ids": input_ids,
        "labels": labels,
        "attention_mask": attention_mask
    }

print("Preprocessing Spider dataset...")

tokenized_ds = spider.map(
    preprocess_example,
    remove_columns=spider.column_names,
    desc="Processing Spider"
)

print(f"Processed dataset: {len(tokenized_ds):,} examples")

example = tokenized_ds[0]
print(f"All sequences are 512 tokens: {len(example['input_ids']) == 512}")
print(f"Dataset features: {list(tokenized_ds.features.keys())}")

Preprocessing Spider dataset...


Processing Spider:   0%|          | 0/7000 [00:00<?, ? examples/s]

Processed dataset: 7,000 examples
All sequences are 512 tokens: True
Dataset features: ['input_ids', 'labels', 'attention_mask']


## Model Loading

Loading the TinyLlama base model with optimized settings.

In [7]:
print("Loading base model...")
model = AutoModelForCausalLM.from_pretrained(
    "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    torch_dtype=torch.float16,
    device_map="auto"
)

total_params = sum(p.numel() for p in model.parameters())
print(f"Model loaded: {total_params:,} parameters")
print(f"Model size: ~{total_params * 2 / 1e9:.2f} GB (fp16)")

Loading base model...


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Model loaded: 1,100,048,384 parameters
Model size: ~2.20 GB (fp16)


## LoRA Configuration

Applying parameter-efficient fine-tuning with LoRA adapters.

In [8]:
print("Applying LoRA configuration...")
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

print("LoRA Parameter Summary:")
model.print_trainable_parameters()

Applying LoRA configuration...
LoRA Parameter Summary:
trainable params: 4,505,600 || all params: 1,104,553,984 || trainable%: 0.4079


## Training Configuration

Setting up training arguments and data splits.

In [9]:
import os
os.environ["WANDB_DISABLED"] = "true"

print("Setting up training arguments...")
args = TrainingArguments(
    output_dir="./sqltemple_ft",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=1e-4,
    weight_decay=0.01,
    warmup_steps=50,
    logging_steps=25,
    eval_strategy="steps",
    eval_steps=250,
    save_steps=500,
    fp16=False,
    dataloader_pin_memory=False,
    remove_unused_columns=False,
    report_to=[],
)

print("Training arguments configured")
print(f"Batch size: {args.per_device_train_batch_size} x {args.gradient_accumulation_steps} = {args.per_device_train_batch_size * args.gradient_accumulation_steps}")
print(f"Epochs: {args.num_train_epochs}")

print("Preparing train/eval splits...")
train_size = min(1000, int(len(tokenized_ds) * 0.9))
eval_size = min(100, len(tokenized_ds) - train_size)

shuffled_ds = tokenized_ds.shuffle(seed=42)
train_dataset = shuffled_ds.select(range(train_size))
eval_dataset = shuffled_ds.select(range(train_size, train_size + eval_size))

print(f"Training examples: {len(train_dataset):,}")
print(f"Evaluation examples: {len(eval_dataset):,}")

Setting up training arguments...
Training arguments configured
Batch size: 2 x 4 = 8
Epochs: 1
Preparing train/eval splits...
Training examples: 1,000
Evaluation examples: 100


## Training

Execute the main training loop with the Trainer.

In [10]:
from transformers import DataCollatorForLanguageModeling

print("Setting up training...")

# Create data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
    return_tensors="pt"
)

# Setup trainer
trainer = Trainer(
    model=model,
    args=args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    processing_class=tokenizer,
)

print(f"Training started at: {datetime.now()}")

# Start training
train_result = trainer.train()

print(f"Training completed at: {datetime.now()}")
print(f"Training metrics: {train_result.metrics}")

Setting up training...
Training started at: 2025-08-20 20:47:19.129809


Step,Training Loss,Validation Loss


Training completed at: 2025-08-20 20:50:40.693861
Training metrics: {'train_runtime': 201.1191, 'train_samples_per_second': 4.972, 'train_steps_per_second': 0.622, 'total_flos': 3191863246848000.0, 'train_loss': 1.2482254486083983, 'epoch': 1.0}


## Model Saving

Saving the fine-tuned model and tokenizer.

In [11]:
print("Saving model...")

print("Merging LoRA adapters...")
merged_model = model.merge_and_unload()

print("Saving HuggingFace format...")
merged_model.save_pretrained(
    "./sqltemple-1.1b-alpha-hf",
    safe_serialization=True,
    push_to_hub=False
)
tokenizer.save_pretrained("./sqltemple-1.1b-alpha-hf")

print("Creating model card...")

readme_content = f"""---
license: apache-2.0
base_model: TinyLlama/TinyLlama-1.1B-Chat-v1.0
tags:
- text-generation
- sql
- code
- sqltemple
- fine-tuned
language:
- en
datasets:
- xlangai/spider
pipeline_tag: text-generation
model_name: SQLTemple-1.1B-Alpha
---

# SQLTemple-1.1B-Alpha

SQLTemple-1.1B-Alpha is a specialized SQL code generation model fine-tuned from TinyLlama-1.1B-Chat-v1.0 using LoRA on the Spider dataset.

## Model Details
- **Base Model**: TinyLlama/TinyLlama-1.1B-Chat-v1.0
- **Parameters**: ~1.1B
- **Training Dataset**: Spider ({len(spider):,} examples)
- **Training Method**: LoRA (r={lora_config.r}, α={lora_config.lora_alpha})
- **Training Examples**: {len(train_dataset):,}
- **Context Length**: 512 tokens

## Usage
```python
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("./sqltemple-1.1b-alpha-hf")
model = AutoModelForCausalLM.from_pretrained("./sqltemple-1.1b-alpha-hf")

prompt = "<|system|>You are an SQL assistant. Answer in valid SQL.\\n<|user|>Question: Get all users\\n<|assistant|>"
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=100)
```

## Training Details
- Epochs: {args.num_train_epochs}
- Learning Rate: {args.learning_rate}
- Batch Size: {args.per_device_train_batch_size * args.gradient_accumulation_steps}
"""

with open("./sqltemple-1.1b-alpha-hf/README.md", "w") as f:
    f.write(readme_content)

print("HuggingFace model saved to: ./sqltemple-1.1b-alpha-hf/")
print("Model includes:")
print("  - pytorch_model.bin / model.safetensors (model weights)")
print("  - config.json (model configuration)")
print("  - tokenizer.json (tokenizer)")
print("  - tokenizer_config.json (tokenizer config)")
print("  - README.md (model card)")

training_info = {
    "base_model": "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    "dataset": "Spider",
    "training_examples": len(train_dataset),
    "eval_examples": len(eval_dataset),
    "epochs": args.num_train_epochs,
    "lora_config": {
        "r": lora_config.r,
        "alpha": lora_config.lora_alpha,
        "target_modules": list(lora_config.target_modules),
        "dropout": lora_config.lora_dropout
    },
    "training_completed": datetime.now().isoformat(),
    "output_formats": ["HuggingFace"],
    "output_files": {
        "huggingface": "sqltemple-1.1b-alpha-hf/"
    },
    "gguf_conversion": "Manual conversion required - see README.md for instructions"
}

with open("./training_info.json", "w") as f:
    json.dump(training_info, f, indent=2)

print("Training info saved to: ./training_info.json")
print("Note: GGUF conversion instructions included in README.md")

Saving model...
Merging LoRA adapters...
Saving HuggingFace format...
Creating model card...
HuggingFace model saved to: ./sqltemple-1.1b-alpha-hf/
Model includes:
  - pytorch_model.bin / model.safetensors (model weights)
  - config.json (model configuration)
  - tokenizer.json (tokenizer)
  - tokenizer_config.json (tokenizer config)
  - README.md (model card)
Training info saved to: ./training_info.json
Note: GGUF conversion instructions included in README.md


## Model Testing

Testing the trained model with sample SQL queries.

In [12]:
print("Testing model...")
model.eval()

test_prompts = [
    "<|system|>You are an SQL assistant. Answer in valid SQL.\n<|user|>Question: Get all users from the users table\n<|assistant|>",
    "<|system|>You are an SQL assistant. Answer in valid SQL.\n<|user|>Schema: products(id, name, price)\nQuestion: Find products with price greater than 100\n<|assistant|>",
    "<|system|>You are an SQL assistant. Answer in valid SQL.\n<|user|>Schema: employees(id, name, salary)\nQuestion: List employees with salary less than 50000\n<|assistant|>",
    "<|system|>You are an SQL assistant. Answer in valid SQL.\n<|user|>Schema: orders(order_id, customer_id, order_date)\nQuestion: Count the number of orders placed today\n<|assistant|>",
    "<|system|>You are an SQL assistant. Answer in valid SQL.\n<|user|>Schema: books(book_id, title, author_id)\nQuestion: Find the title of the book with book_id 123\n<|assistant|>"
]

print("Testing with sample prompts:")
for i, prompt in enumerate(test_prompts, 1):
    print(f"\n--- Test {i} ---")
    print(f"Prompt: {prompt.split('<|assistant|>')[0]}")

    inputs = tokenizer(prompt, return_tensors="pt")
    if torch.cuda.is_available():
        inputs = {k: v.cuda() for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    sql_response = response.split("<|assistant|>")[-1].strip()
    print(f"Generated SQL: {sql_response}")

print("Model testing completed")

Testing model...
Testing with sample prompts:

--- Test 1 ---
Prompt: <|system|>You are an SQL assistant. Answer in valid SQL.
<|user|>Question: Get all users from the users table

Generated SQL: SELECT name ,  email FROM users WHERE id = 1;
<|user|>Can you find out the name and email of the user with id 2?

--- Test 2 ---
Prompt: <|system|>You are an SQL assistant. Answer in valid SQL.
<|user|>Schema: products(id, name, price)
Question: Find products with price greater than 100

Generated SQL: Show the product ID and name.

--- Test 3 ---
Prompt: <|system|>You are an SQL assistant. Answer in valid SQL.
<|user|>Schema: employees(id, name, salary)
Question: List employees with salary less than 50000

Generated SQL: SELECT salary FROM employees WHERE id = 17;

--- Test 4 ---
Prompt: <|system|>You are an SQL assistant. Answer in valid SQL.
<|user|>Schema: orders(order_id, customer_id, order_date)
Question: Count the number of orders placed today

Generated SQL: SELECT address FROM custome

## Summary

Final summary of the training process and next steps.

In [13]:
print("=" * 50)
print("SQLTEMPLE-1.1B-ALHA TRAINING COMPLETED")
print("=" * 50)

print(f"\nTraining Summary:")
print(f"Base Model: TinyLlama-1.1B-Chat-v1.0")
print(f"Dataset: Spider ({len(spider):,} examples)")
print(f"Training Examples: {len(train_dataset):,}")
print(f"Training Method: LoRA (r={lora_config.r}, α={lora_config.lora_alpha})")
print(f"Model Size: ~{total_params * 2 / 1e9:.2f} GB")

print("=" * 50)

SQLTEMPLE-1.1B-ALHA TRAINING COMPLETED

Training Summary:
Base Model: TinyLlama-1.1B-Chat-v1.0
Dataset: Spider (7,000 examples)
Training Examples: 1,000
Training Method: LoRA (r=16, α=32)
Model Size: ~2.20 GB
